## Import Dependencies
In this project, main libraries we used are:
* Tensorflow 
* Numpy
* os
* PIL
* Matplotlib

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
# Check tensorflow version
print(tf.__version__)

## Resizing
Here we resized all of the image in a folder to 200x200 pixels. 
Not that we did this per folder (train and validation), ensuring every image is intact and not corrupted. 

Our advice is to check every folder to see if there are some corrupted images after resizing, then delete and/or change it immediately.

In [ ]:
import os
from PIL import Image

f = r'C:\Users\Asus\Downloads\Compressed\NEW-BATIK-DATASET\prg'
    
new_d = 200

for file in os.listdir(f):
    f_img = f+'/'+file
    try:
        img = Image.open(f_img)
        img = img.resize((new_d, new_d))
        img.save(f_img)
    except IOError:
        pass

## Load Image Sources
Here we are using **ImageDataGenerator** as a loader for images in our directory.

We implemented Image augmentation in the training directory to prevent overfitting. We also load data locally, since we were using Jupyter Notebook with Tensorflow 2.4.1

In [ ]:
# Load Training 
TRAINING_DIR = "C:\\Users\\Asus\\Downloads\\Compressed\\cobatrain"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=45,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Load Validation
VALIDATION_DIR = "C:\\Users\\Asus\\Downloads\\Compressed\\cobaval"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(200,200),
    class_mode='categorical',
    #Expreimented with batch size
    batch_size=6)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(200,200),
    class_mode='categorical',
    #Same here
    batch_size=5)

## Creating Keras Model
Now that we loaded the images, we create our model using **Convolutional** method.
We used 5 Neurons in our final **Dense** layer, because we are identifying for 5 classes.

Then, we compile our model with **accuracy** as the metric.

In [ ]:
# Convolutional Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Using adam optimizer for faster training
model.compile(loss = 'categorical_crossentropy',
          optimizer='adam',
          metrics=['acc'])

In [ ]:
# Inspect Model
model.summary()

## Run the Model
Note that we run the model with 200 epochs, we did this because the dataset itself is pretty noisy and requires large epoch for more accuracy.

In [ ]:
history = model.fit(train_generator,
                    epochs=200,
                    # Experiment with steps
                    steps_per_epoch=6,
                    validation_data = validation_generator,
                    shuffle=True,
                    verbose = 1)

In [ ]:
# Create a Graph from history object

%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

## Try Model
And we're done! It's time to check our model.
Note that this model expects the input to be 200x200 pixels, and so before importing we need to resize input image first.

In [ ]:
# First load dependencies
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

In [ ]:
# Load singular image
img = image.load_img('C:\\Users\\Asus\\Downloads\\motifBatikbaru.jpg', target_size=(200,200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [ ]:
# Pass image to model.predict
images = np.vstack([x])
classes = np.argmax(model.predict(images))
if classes == [0]:
    print("Batik Celup")
elif classes == [1]:
    print("Batik Kawung")
elif classes == [2]:
    print("Batik Megamendung")
elif classes == [3]:
    print("Batik Parang")
else:
    print("Batik Sidoluhur")
print(classes)

In [ ]:
# Save Model
model.save('C:\\Users\\Asus\\Downloads\\saved models\\savedmodel200v2')

## Final notes
As mentioned many, many times, the dataset itself is very noisy. Batik itself is a complex form of art, and as a cultural representative to numerous regions in Indonesia. And so some regions may have very similar pattern even though the name is different.

In conclusion, this model can be improved, and the dataset can be improved too. The variety of batik images shouldn't stop our progress to raise awareness for batik types in Indonesia. Thank you!